In [ ]:
import psycopg2
import pandas as pd
import os
import itertools
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
conn = psycopg2.connect(
    host='localhost',
    port=54320,
    dbname='my_database',
    password='pass',
    user='user',
    )

In [ ]:
def get_ordered_users_with_roles():
    cur = conn.cursor()
    query = """
    SELECT user_id, label, date
    FROM user_mapping
    ORDER BY user_id, date
    """
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
user_mappings = get_ordered_users_with_roles()

In [ ]:
df = pd.DataFrame(user_mappings, columns = ["user_id", "label", "date"] )

In [ ]:
def count_stability(df):
    leng = len(df)
    
    first_row = df.iloc[0]
    tail = df[1:]
    changes = {}
    stability = {}
    count = 1
    for index, row in tail.iterrows():
        
        if index % 100000 == 0:
            print(str(index /leng))
        
        if first_row["user_id"] == row["user_id"]:
            keyChange = first_row["label"] + " -> " + row["label"]
            keyStability = first_row["label"] + " -> " + first_row["label"]
        
            if first_row["label"] != row["label"]:
            
                if stability.get(keyStability):
                    stability.get(keyStability).append(count)
                else:
                    stability[keyStability] = [count]
                
            
                if changes.get(keyChange):
                    changes[keyChange] = changes.get(keyChange) + 1
                else:
                    changes[keyChange] = 1
                
                count = 1
            else:
                count = count + 1
        else:
            count = 1
            
        first_row = row
    return changes, stability



In [ ]:
changes, stability = count_stability(df)

In [ ]:
changes_df = pd.DataFrame(changes, index=[0]).T

In [ ]:
changes_df_sorted = changes_df.sort_values(0, ascending=False)

In [ ]:
changes_df_sorted.columns = ["number of changes"]

In [ ]:
changes_df_sorted # moze warto dodac tu procentowo ile to bylo zmian czyli 3 -> 0 = ilosc zmian / ilosc klastrow 3 *100

In [ ]:
stability_stats = pd.DataFrame()
for key in stability:
    stability_stats[key] = pd.Series(stability.get(key)).describe()

In [ ]:
stability_stats[1:]

### Changes in influential roles

In [ ]:
def flat(list2):
    return [e[0] for e in list2]

In [ ]:
def get_all_users_with_role(role):
    cur = conn.cursor()
    query = f"""
    SELECT DISTINCT user_id
    FROM user_mapping
    WHERE label = '{role}'
    """
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
users_with_role_6 = get_all_users_with_role("6")

In [ ]:
users_with_role_6 = flat(users_with_role_6)

In [ ]:
users_with_role_6

In [ ]:
def get_all_dates():
    cur = conn.cursor()
    query = f"""
    SELECT DISTINCT date 
    FROM user_mapping 
    WHERE user_id = '625'
    ORDER BY date;
    """
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
dates = pd.DataFrame(get_all_dates(), columns=["date"])

In [ ]:
def get_all_labels_for_user(user_id):
    cur = conn.cursor()
    query = f"""
    SELECT date, label
    FROM user_mapping 
    WHERE user_id = '{user_id}'
    ORDER BY date;
    """
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
user_labels = get_all_labels_for_user(856)

In [ ]:
user_labels_df = pd.DataFrame(user_labels, columns=["date", "label" ])

In [ ]:
user_labels_df

In [ ]:
pd.merge(dates, user_labels_df, on='date', how='left')

In [ ]:
def prepare_slots_for_users_with_label(label):
    base = pd.DataFrame(get_all_dates(), columns=["date"])
    users_with_role = flat(get_all_users_with_role(label))
    for user_id in users_with_role:
        user_labels = get_all_labels_for_user(user_id)
        user_labels_df = pd.DataFrame(user_labels, columns=["date", "label" ])
        base = pd.merge(base, user_labels_df, on='date', how='left')
    base = base.fillna(-1)
    columns = ["date"]
    columns.extend(users_with_role)
    base.columns = columns
    
    conv_dict = { feat : int for feat in users_with_role }
    
    base = base.astype(conv_dict)
    
    plt.figure(figsize=(20,10))
    for user_id in users_with_role:
        plt.plot(base.index, base[user_id])
        
    plt.title("Roles of users who were at least once in role of {}".format(label))
    plt.ylabel("Role number")
    plt.xlabel("Slot number")
    plt.legend()
    plt.show()
    return base

In [ ]:
slots_df = prepare_slots_for_users_with_label(6)

In [ ]:
slots_df = prepare_slots_for_users_with_label(1)

In [ ]:
slots_df = prepare_slots_for_users_with_label(4)

In [ ]:
user_mappings = get_ordered_users_with_roles()

In [ ]:
df = pd.DataFrame(user_mappings, columns = ["user_id", "label", "date"] )

In [ ]:
len(df["user_id"].unique())

In [ ]:
start_date = datetime.date(2009,12,22)
end_date = start_date + timedelta((2 - 1)*14)

In [ ]:
mappings_for_n = df[(df['date'] >= str(start_date)) & (df['date'] <= str(end_date)) & (df['user_id'] == 590)]

In [ ]:
mappings_for_n

In [ ]:
l = df[5:10]

In [ ]:
df.loc[590]

In [ ]:
for index, row in l.iterrows():
    print(row["label"])

In [ ]:
def is_the_same(mappings, n):
    if len(mappings) < n:
        return False
    
    if n == 0:
        return True
    
    first_row = mappings.iloc[0]
    tail = mappings[1:]
    
    for index, row in tail.iterrows():
        if row["user_id"] != first_row["user_id"]:
            return False
        if row["label"] != first_row["label"]:
            return False
        first_row = row
    return True

In [ ]:
is_the_same(l, 5)

In [ ]:
import datetime
from datetime import timedelta 

In [ ]:
def count_numer_of_users_in_roles_for(slot_nums, user_mappings, role):
    result = []
    user_mappings = user_mappings[(user_mappings['label'] == str(role))]
    
    start_date = datetime.date(2009,12,22)
    end_date = start_date + timedelta((slot_nums - 1)*14)
    final_date = datetime.date(2013,10, 8)
    
    
    list_of_users = user_mappings["user_id"].unique()
    while(start_date <= final_date):
        count = 0
        for user in list_of_users:
            mappings_for_n = user_mappings[(user_mappings['date'] >= str(start_date)) & (user_mappings['date'] <= str(end_date)) & (user_mappings['user_id'] == user)]
            
            if is_the_same(mappings_for_n, slot_nums):
                count+=1
        
        result.append(count)
        start_date += timedelta(14)
        end_date += timedelta(14)
    print(f"Role {role} with slot_nums: {slot_nums} is finished")
    return result
    

In [ ]:
user_mappings = get_ordered_users_with_roles()

In [ ]:
df_stability = pd.DataFrame(user_mappings, columns = ["user_id", "label", "date"] )

In [ ]:
def plot_stability(role, df):
    dict_of_stab = {
            'the_same_role_for_2_slots': count_numer_of_users_in_roles_for(1, df, role),
            'the_same_role_for_3_slots': count_numer_of_users_in_roles_for(2, df, role),
            'the_same_role_for_4_slots': count_numer_of_users_in_roles_for(3, df, role),
            'the_same_role_for_5_slots': count_numer_of_users_in_roles_for(4, df, role),
            'the_same_role_for_6_slots': count_numer_of_users_in_roles_for(5, df, role),
            'the_same_role_for_7_slots': count_numer_of_users_in_roles_for(6, df, role)
    }
        
    return pd.DataFrame.from_dict(dict_of_stab)
        

    




In [ ]:
role = 6
to_plot = plot_stability(role, df_stability)
to_plot.plot.line(figsize=(20,10),title="Stability for role {}".format(role))

In [ ]:
role = 4
to_plot = plot_stability(role, df_stability)
to_plot.plot.line(figsize=(20,10),title="Stability for role {}".format(role))

In [ ]:
role = 1
to_plot = plot_stability(role, df_stability)
to_plot.plot.line(figsize=(20,10),title="Stability for role {}".format(role))

In [ ]:
role = 2
to_plot = plot_stability(role, df_stability)
to_plot.plot.line(figsize=(20,10),title="Stability for role {}".format(role))

In [ ]:
role = 5
to_plot = plot_stability(role, df_stability)
to_plot.plot.line(figsize=(20,10),title="Stability for role {}".format(role))

In [ ]:
role = 3
to_plot = plot_stability(role, df_stability)
to_plot.plot.line(figsize=(20,10),title="Stability for role {}".format(role))

In [ ]:
role = 1
to_plot = plot_stability(role, df_stability)
to_plot.plot.line(figsize=(20,10),title="Stability for role {}".format(role))

In [ ]:
user_mappings = get_ordered_users_with_roles()

In [ ]:
df = pd.DataFrame(user_mappings, columns = ["user_id", "label", "date"] )

In [ ]:
def count_stability_with_break(df):
    leng = len(df)
    
    first_row = df.iloc[0]
    second_row = df.iloc[1]
    tail = df[2:]
    changes = {}
    stability = {}
    breaks = {}
    count = 1
    break_num = 0
    
    for index, row in tail.iterrows():
        
        if index % 100000 == 0:
            print(str(index /leng))
        
        if first_row["user_id"] == second_row["user_id"] and first_row["user_id"] == row["user_id"]:
            #keyChange = first_row["label"] + " -> " + row["label"]
            keyStability = first_row["label"] + " -> " + first_row["label"]
        
            # dwa kolejne sa rozne od pierwszego 
            if first_row["label"] != second_row["label"] and first_row["label"] != row["label"]:
            
                if stability.get(keyStability):
                    stability.get(keyStability).append(count)
                else:
                    stability[keyStability] = [count]
                
                #liczone przerwy tylko jezeli sa 3 slotowe ciagi niezmienne 
                if count >= 3:
                    if breaks.get(keyStability):
                        breaks.get(keyStability).append(break_num)
                    else:
                        breaks[keyStability] = [break_num]
                
            
#                 if changes.get(keyChange):
#                     changes[keyChange] = changes.get(keyChange) + 1
#                 else:
#                     changes[keyChange] = 1

                count = 1
            else:
                count = count + 1
            
            ### Number of breaks
            
            if (first_row["label"] != second_row["label"] or second_row["label"] != row["label"]):     
                break_num = break_num + 1
                
            
        else:
            keyStability = first_row["label"] + " -> " + first_row["label"]
            #add last result 
            if stability.get(keyStability):
                stability.get(keyStability).append(count)
            else:
                stability[keyStability] = [count]
                
            count = 1
            break_num = 0
            
        first_row = second_row
        second_row = row
    return breaks, stability

In [ ]:
breaks, stability = count_stability_with_break(df)

In [ ]:
stability_stats = pd.DataFrame()
for key in stability:
    stability_stats[key] = pd.Series(stability.get(key)).describe()

In [ ]:
breaks_stats = pd.DataFrame()
for key in breaks:
    breaks_stats[key] = pd.Series(breaks.get(key)).describe()

In [ ]:
stability_stats[1:]

In [ ]:
breaks_stats[1:]

In [ ]:
def count_stability_with_all_kind_of_breaks(df):
    leng = len(df)
    
    first_row = df.iloc[0]
    tail = df[1:]
    roles = {"0":0, "1":0, "2":0, "3":0, "4":0, "5":0, "6":0}
    stability = {}
    count = 1
    for index, row in tail.iterrows():
        
        if index % 100000 == 0:
            print(str(index /leng))
        
        if first_row["user_id"] == row["user_id"]:
            keyStability = first_row["label"]
            
        
            if first_row["label"] != row["label"]:
            
                if stability.get(keyStability):
                    stability.get(keyStability).append(count)
                else:
                    stability[keyStability] = [count]
                
            
                if changes.get(keyChange):
                    changes[keyChange] = changes.get(keyChange) + 1
                else:
                    changes[keyChange] = 1
                
                count = 1
            else:
                count = count + 1
        else:
            count = 1
            
        first_row = row
    return changes, stability

In [ ]:
def get_general_roles_statistics():
    cur = conn.cursor()
    query = """
    WITH 
    general_stats AS (SELECT COUNT(1) as number_of_slots, label
    FROM user_mapping
    GROUP BY user_id, label)
    
    SELECT AVG(number_of_slots), stddev(number_of_slots),
    percentile_cont(0.25) WITHIN GROUP (ORDER BY number_of_slots),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY number_of_slots),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY number_of_slots),
    MAX(number_of_slots), MIN(number_of_slots) FROM general_stats
    GROUP BY label
    """
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
gen_stats = get_general_roles_statistics()

In [ ]:
general_roles_statistics = pd.DataFrame(gen_stats, columns = ["mean", "stddev", "25%", "50%", "75%", "max", "min"] )

In [ ]:
general_roles_statistics

#### for n breaks

In [ ]:
user_mappings = get_ordered_users_with_roles()

In [ ]:
df = pd.DataFrame(user_mappings, columns = ["user_id", "label", "date"] )

In [ ]:
def is_break_greater(df, index, n, length):
    if index + n < length:
        n = length - index - 1
    for i in range(1, n + 1):
        if df.iloc[index]["label"] == df.iloc[index + i]["label"]:
            return False
    return True

In [ ]:
def is_user_the_same(df, index, n, length):
    if index + n < length:
        n = length - index - 1
    for i in range(1, n + 1):
        if df.iloc[index]["user_id"] != df.iloc[index + i]["user_id"]:
            return False
    return True

In [ ]:
def count_stability_with_break(df, breaks):
    leng = len(df)
    
    first_row = df.iloc[0]
    #second_row = df.iloc[1]
    tail = df[1:]
    changes = {}
    stability = {}
    count = 1
    break_num = 0
    
    for index, row in tail.iterrows():
        
        if index % 100000 == 0:
            print(str(index /leng))
        
        if is_user_the_same(df, index, breaks, leng):
            #keyChange = first_row["label"] + " -> " + row["label"]
            keyStability = first_row["label"] + " -> " + first_row["label"]
        
            # dwa kolejne sa rozne od pierwszego 
            if is_break_greater(df, index - 1, breaks, leng):
            
                if stability.get(keyStability):
                    stability.get(keyStability).append(count)
                else:
                    stability[keyStability] = [count]
                
                #liczone przerwy tylko jezeli sa 3 slotowe ciagi niezmienne 
#                 if count >= 3:
#                     if breaks.get(keyStability):
#                         breaks.get(keyStability).append(break_num)
#                     else:
#                         breaks[keyStability] = [break_num]
                
            
#                 if changes.get(keyChange):
#                     changes[keyChange] = changes.get(keyChange) + 1
#                 else:
#                     changes[keyChange] = 1

                count = 1
            else:
                count = count + 1
            
            ### Number of breaks
            
#             if (first_row["label"] != second_row["label"] or second_row["label"] != row["label"]):     
#                 break_num = break_num + 1
                
            
        else:
            keyStability = first_row["label"] + " -> " + first_row["label"]
            #add last result
            
            if stability.get(keyStability):
                stability.get(keyStability).append(count)
            else:
                stability[keyStability] = [count]
                
            count = 1
            break_num = 0
            
        first_row = row
    return breaks, stability

In [ ]:
breaks, stability = count_stability_with_break(df, 2)

In [ ]:
stability_stats = pd.DataFrame()
for key in stability:
    stability_stats[key] = pd.Series(stability.get(key)).describe()

In [ ]:
breaks_stats = pd.DataFrame()
for key in breaks:
    breaks_stats[key] = pd.Series(breaks.get(key)).describe()

In [ ]:
stability_stats[1:]

In [ ]:
breaks, stability = count_stability_with_break(df, 3)

In [ ]:
stability_stats = pd.DataFrame()
for key in stability:
    stability_stats[key] = pd.Series(stability.get(key)).describe()

In [ ]:
stability_stats[1:]

#### for 2 breaks

In [ ]:
def count_stability_with_break(df):
    leng = len(df)
    
    first_row = df.iloc[0]
    second_row = df.iloc[1]
    third_row = df.iloc[2]
    tail = df[3:]
    changes = {}
    stability = {}
    breaks = {}
    count = 1
    break_num = 0
    
    for index, row in tail.iterrows():
        
        if index % 100000 == 0:
            print(str(index /leng))
        
        if first_row["user_id"] == second_row["user_id"] and first_row["user_id"] == third_row["user_id"] and first_row["user_id"] == row["user_id"]:
            #keyChange = first_row["label"] + " -> " + row["label"]
            keyStability = first_row["label"] + " -> " + first_row["label"]
        
            # dwa kolejne sa rozne od pierwszego 
            if first_row["label"] != second_row["label"] and first_row["label"] != third_row["label"] and first_row["label"] != row["label"]:
            
                if stability.get(keyStability):
                    stability.get(keyStability).append(count)
                else:
                    stability[keyStability] = [count]
                
                #liczone przerwy tylko jezeli sa 3 slotowe ciagi niezmienne 
                if count >= 3:
                    if breaks.get(keyStability):
                        breaks.get(keyStability).append(break_num)
                    else:
                        breaks[keyStability] = [break_num]
                
            
#                 if changes.get(keyChange):
#                     changes[keyChange] = changes.get(keyChange) + 1
#                 else:
#                     changes[keyChange] = 1

                count = 1
            else:
                count = count + 1
            
            ### Number of breaks
            
            if (first_row["label"] != second_row["label"] or second_row["label"] != row["label"]):     
                break_num = break_num + 1
                
            
        else:
            keyStability = first_row["label"] + " -> " + first_row["label"]
            #add last result 
            if stability.get(keyStability):
                stability.get(keyStability).append(count)
            else:
                stability[keyStability] = [count]
                
            count = 1
            break_num = 0
            
        first_row = second_row
        second_row = third_row
        third_row = row
    return breaks, stability

In [ ]:
breaks, stability = count_stability_with_break(df)

In [ ]:
stability_stats = pd.DataFrame()
for key in stability:
    stability_stats[key] = pd.Series(stability.get(key)).describe()

In [ ]:
stability_stats[1:]

In [ ]:
pd.Series(stability.get('0 -> 0')).describe()

#### for 3 breaks

In [ ]:
def count_stability_with_break(df):
    leng = len(df)
    
    first_row = df.iloc[0]
    second_row = df.iloc[1]
    third_row = df.iloc[2]
    forth_row = df.iloc[3]
    tail = df[4:]
    changes = {}
    stability = {}
    breaks = {}
    count = 1
    break_num = 0
    
    for index, row in tail.iterrows():
        
        if index % 100000 == 0:
            print(str(index /leng))
        
        if first_row["user_id"] == second_row["user_id"] and first_row["user_id"] == third_row["user_id"] and first_row["user_id"] == forth_row["user_id"] and first_row["user_id"] == row["user_id"]:
            #keyChange = first_row["label"] + " -> " + row["label"]
            keyStability = first_row["label"] + " -> " + first_row["label"]
        
            # dwa kolejne sa rozne od pierwszego 
            if first_row["label"] != second_row["label"] and first_row["label"] != third_row["label"] and first_row["label"] != first_row["label"] and first_row["label"] != row["label"]:
            
                if stability.get(keyStability):
                    stability.get(keyStability).append(count)
                else:
                    stability[keyStability] = [count]
                
                #liczone przerwy tylko jezeli sa 3 slotowe ciagi niezmienne 
                if count >= 3:
                    if breaks.get(keyStability):
                        breaks.get(keyStability).append(break_num)
                    else:
                        breaks[keyStability] = [break_num]
                
            
#                 if changes.get(keyChange):
#                     changes[keyChange] = changes.get(keyChange) + 1
#                 else:
#                     changes[keyChange] = 1

                count = 1
            else:
                count = count + 1
            
            ### Number of breaks
            
            if (first_row["label"] != second_row["label"] or second_row["label"] != row["label"]):     
                break_num = break_num + 1
                
            
        else:
            keyStability = first_row["label"] + " -> " + first_row["label"]
            #add last result 
            if stability.get(keyStability):
                stability.get(keyStability).append(count)
            else:
                stability[keyStability] = [count]
                
            count = 1
            break_num = 0
            
        first_row = second_row
        second_row = third_row
        third_row = row
    return breaks, stability

In [ ]:
breaks, stability = count_stability_with_break(df)

In [ ]:
stability_stats = pd.DataFrame()
for key in stability:
    stability_stats[key] = pd.Series(stability.get(key)).describe()

In [ ]:
stability_stats[1:]

#### Influential users

#### Active Blogger

In [ ]:
def get_active_blogger_statistics():
    cur = conn.cursor()
    query = """
    WITH
    general_stats AS (
    SELECT
    user_id,
    SUM (CASE WHEN label = '0' THEN 1 ELSE 0 END) AS number_of_slots_as_active_commentators,
    SUM (CASE WHEN label = '1' THEN 1 ELSE 0 END) AS number_of_slots_as_active_bloggers,
    SUM (CASE WHEN label = '2' THEN 1 ELSE 0 END) AS number_of_slots_as_common_bloggers,
    SUM (CASE WHEN label = '3' THEN 1 ELSE 0 END) AS number_of_slots_as_low_active_commentators,
    SUM (CASE WHEN label = '4' THEN 1 ELSE 0 END) AS number_of_slots_as_influential_bloggers,
    SUM (CASE WHEN label = '5' THEN 1 ELSE 0 END) AS number_of_slots_as_engaged_bloggers,
    SUM (CASE WHEN label = '6' THEN 1 ELSE 0 END) AS number_of_slots_as_authority_bloggers
    FROM user_mapping
    GROUP BY user_id)
    
    SELECT *
    FROM general_stats
    WHERE number_of_slots_as_active_bloggers != 0
    """
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
active_blogger_results = get_active_blogger_statistics()

In [ ]:
results

In [ ]:
active_blogger_statistics = pd.DataFrame(active_blogger_results, columns = ["user_id", "0", "1", "2", "3", "4", "5", "6"] )

In [ ]:
active_blogger_statistics.drop(columns = ["user_id"]).describe()[1:]

In [ ]:
def get_influential_bloggers_statistics():
    cur = conn.cursor()
    query = """
    WITH
    general_stats AS (
    SELECT
    user_id,
    SUM (CASE WHEN label = '0' THEN 1 ELSE 0 END) AS number_of_slots_as_active_commentators,
    SUM (CASE WHEN label = '1' THEN 1 ELSE 0 END) AS number_of_slots_as_active_bloggers,
    SUM (CASE WHEN label = '2' THEN 1 ELSE 0 END) AS number_of_slots_as_common_bloggers,
    SUM (CASE WHEN label = '3' THEN 1 ELSE 0 END) AS number_of_slots_as_low_active_commentators,
    SUM (CASE WHEN label = '4' THEN 1 ELSE 0 END) AS number_of_slots_as_influential_bloggers,
    SUM (CASE WHEN label = '5' THEN 1 ELSE 0 END) AS number_of_slots_as_engaged_bloggers,
    SUM (CASE WHEN label = '6' THEN 1 ELSE 0 END) AS number_of_slots_as_authority_bloggers
    FROM user_mapping
    GROUP BY user_id)
    
    SELECT *
    FROM general_stats
    WHERE number_of_slots_as_influential_bloggers != 0
    """
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
influential_bloggers_results = get_influential_bloggers_statistics()

In [ ]:
influential_bloggers_statistics = pd.DataFrame(influential_bloggers_results, columns = ["user_id", "0", "1", "2", "3", "4", "5", "6"] )

In [ ]:
influential_bloggers_statistics.drop(columns = ["user_id"]).describe()[1:]

In [ ]:
def get_authority_bloggers_statistics():
    cur = conn.cursor()
    query = """
    WITH
    general_stats AS (
    SELECT
    user_id,
    SUM (CASE WHEN label = '0' THEN 1 ELSE 0 END) AS number_of_slots_as_active_commentators,
    SUM (CASE WHEN label = '1' THEN 1 ELSE 0 END) AS number_of_slots_as_active_bloggers,
    SUM (CASE WHEN label = '2' THEN 1 ELSE 0 END) AS number_of_slots_as_common_bloggers,
    SUM (CASE WHEN label = '3' THEN 1 ELSE 0 END) AS number_of_slots_as_low_active_commentators,
    SUM (CASE WHEN label = '4' THEN 1 ELSE 0 END) AS number_of_slots_as_influential_bloggers,
    SUM (CASE WHEN label = '5' THEN 1 ELSE 0 END) AS number_of_slots_as_engaged_bloggers,
    SUM (CASE WHEN label = '6' THEN 1 ELSE 0 END) AS number_of_slots_as_authority_bloggers
    FROM user_mapping
    GROUP BY user_id)
    
    SELECT *
    FROM general_stats
    WHERE number_of_slots_as_authority_bloggers != 0
    """
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
authority_bloggers_results = get_authority_bloggers_statistics()

In [ ]:
authority_bloggers_statistics = pd.DataFrame(authority_bloggers_results, columns = ["user_id", "0", "1", "2", "3", "4", "5", "6"] )

In [ ]:
authority_bloggers_statistics.drop(columns = ["user_id"]).describe()[1:]